In [1]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))



In [2]:
def get_pdf_text(pdf_docs):
    text=""
    pdf_reader= PdfReader(pdf_docs)
    for page in pdf_reader.pages:
        text+= page.extract_text()
    return  text



def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks


def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")


In [3]:
raw_text = get_pdf_text(r"E:\unt\spring_2024\info retrieval\project\Project Proposal Guideline.pdf")
raw_text

" \nProject Proposal  Guideline  \nMake sure everyone in your team approves of the proposal. Then, only one member \nneeds to submit the PDF of your proposal to Canvas as described below:  \n \nProject Name, Participants, and Work flow  \nGive the project a good name and roles of participants in as much detail as possible. \nDocument how will you work collaboratively on the project. For example, are you \ngoing to have weekly meetings outside class? When?  \n \nProject Abstract  \nInclude 1 -3 paragraphs explaining your project. This abstract should focus on the \ngoal of the project from a user’s perspective rather than the underlying \nimplementation methods you will use. Describe your target and Actions and  provide \na workflow diagram with explanation.  \n \nData Abstraction  \nList the types and attributes of the dataset, and a detailed description of dataset.  \n \nProject Design  \nList the underlying technologies and design methods in as much detail as possible. \nFor example,

In [4]:
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks)

In [8]:
def get_conversational_chain():

    prompt_template =  """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain


In [42]:
def user_input(user_question):
    PROMPT_TEMPLATE = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    
    new_db = new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
    docs = new_db.similarity_search_with_score(user_question)
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in docs])
    prompt = PROMPT_TEMPLATE.format(context=context_text, question=user_question,return_only_outputs=True)
    print(prompt)
    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)
    chain = load_qa_chain(model, chain_type="stuff")
    answer = chain.run(input_documents=docs, question=user_question)
    print(answer)

In [43]:
user_input("how many members can a team be") 

[Document(page_content="Project Proposal  Guideline  \nMake sure everyone in your team approves of the proposal. Then, only one member \nneeds to submit the PDF of your proposal to Canvas as described below:  \n \nProject Name, Participants, and Work flow  \nGive the project a good name and roles of participants in as much detail as possible. \nDocument how will you work collaboratively on the project. For example, are you \ngoing to have weekly meetings outside class? When?  \n \nProject Abstract  \nInclude 1 -3 paragraphs explaining your project. This abstract should focus on the \ngoal of the project from a user’s perspective rather than the underlying \nimplementation methods you will use. Describe your target and Actions and  provide \na workflow diagram with explanation.  \n \nData Abstraction  \nList the types and attributes of the dataset, and a detailed description of dataset.  \n \nProject Design  \nList the underlying technologies and design methods in as much detail as poss

ValueError: too many values to unpack (expected 2)

In [5]:
user_question="how many members can a team be"
PROMPT_TEMPLATE = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

new_db = new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
retriever = new_db.as_retriever(score_threshold = 0.7)

PROMPT = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
model = ChatGoogleGenerativeAI(model="gemini-pro",
                            temperature=0.6)
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=model,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

chain("how many members can a team be")

c:\Users\jashi\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'how many members can a team be',
 'result': 'Answer is not available in the context',
 'source_documents': [Document(page_content="Project Proposal  Guideline  \nMake sure everyone in your team approves of the proposal. Then, only one member \nneeds to submit the PDF of your proposal to Canvas as described below:  \n \nProject Name, Participants, and Work flow  \nGive the project a good name and roles of participants in as much detail as possible. \nDocument how will you work collaboratively on the project. For example, are you \ngoing to have weekly meetings outside class? When?  \n \nProject Abstract  \nInclude 1 -3 paragraphs explaining your project. This abstract should focus on the \ngoal of the project from a user’s perspective rather than the underlying \nimplementation methods you will use. Describe your target and Actions and  provide \na workflow diagram with explanation.  \n \nData Abstraction  \nList the types and attributes of the dataset, and a detailed descrip

In [6]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain_community.utilities import SerpAPIWrapper

tools=load_tools(["serpapi","llm-math"],llm=model)
agent_exe=initialize_agent(
    tools,
    model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)
agent_exe.run("what is taylor swifts age")

c:\Users\jashi\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Users\jashi\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Action: Search
Action Input: How old is Taylor Swift?
Observation: 34 years
Thought:Final Answer: 34

> Finished chain.


'34'

In [62]:
user_question = "how many members can a team be"
PROMPT_TEMPLATE = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details. If the answer is not in the provided context, just say, "answer is not available in the context". Don't provide the wrong answer.

    Context:
    {context}?

    Question:
    {question}

    Answer:
    """

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(user_question)
print(docs)

context_text = docs[0].page_content
prompt = PROMPT_TEMPLATE.format(context=context_text, question=user_question)

print(prompt)

model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
chain1 = load_qa_chain(model, chain_type="map_reduce")
answer = chain1.run(input_documents=docs, question=user_question)
print(answer)

[Document(page_content="Project Proposal  Guideline  \nMake sure everyone in your team approves of the proposal. Then, only one member \nneeds to submit the PDF of your proposal to Canvas as described below:  \n \nProject Name, Participants, and Work flow  \nGive the project a good name and roles of participants in as much detail as possible. \nDocument how will you work collaboratively on the project. For example, are you \ngoing to have weekly meetings outside class? When?  \n \nProject Abstract  \nInclude 1 -3 paragraphs explaining your project. This abstract should focus on the \ngoal of the project from a user’s perspective rather than the underlying \nimplementation methods you will use. Describe your target and Actions and  provide \na workflow diagram with explanation.  \n \nData Abstraction  \nList the types and attributes of the dataset, and a detailed description of dataset.  \n \nProject Design  \nList the underlying technologies and design methods in as much detail as poss

ValueError: SystemMessages are not yet supported!

To automatically convert the leading SystemMessage to a HumanMessage,
set  `convert_system_message_to_human` to True. Example:

llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)


In [61]:
print(docs[0])
type(docs[0])

page_content="Project Proposal  Guideline  \nMake sure everyone in your team approves of the proposal. Then, only one member \nneeds to submit the PDF of your proposal to Canvas as described below:  \n \nProject Name, Participants, and Work flow  \nGive the project a good name and roles of participants in as much detail as possible. \nDocument how will you work collaboratively on the project. For example, are you \ngoing to have weekly meetings outside class? When?  \n \nProject Abstract  \nInclude 1 -3 paragraphs explaining your project. This abstract should focus on the \ngoal of the project from a user’s perspective rather than the underlying \nimplementation methods you will use. Describe your target and Actions and  provide \na workflow diagram with explanation.  \n \nData Abstraction  \nList the types and attributes of the dataset, and a detailed description of dataset.  \n \nProject Design  \nList the underlying technologies and design methods in as much detail as possible. \nFo

langchain_core.documents.base.Document

In [26]:
def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain



def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    
    new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)
    st.write("Reply: ", response["output_text"])


In [8]:
def main():
    st.set_page_config("Chat PDF")
    st.header("Chat with PDF using Gemini💁")

    user_question = st.text_input("Ask a Question from the PDF Files")

    if user_question:
        user_input(user_question)

    with st.sidebar:
        st.title("Menu:")
        pdf_docs = st.file_uploader("Upload your PDF Files and Click on the Submit & Process Button", accept_multiple_files=True)
        if st.button("Submit & Process"):
            with st.spinner("Processing..."):
                raw_text = get_pdf_text(pdf_docs)
                text_chunks = get_text_chunks(raw_text)
                get_vector_store(text_chunks)
                st.success("Done")



if __name__ == "__main__":
    main()

2024-04-10 01:32:08.465 
  command:

    streamlit run c:\Users\jashi\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [12]:
!streamlit run st.py